In [1]:
import pandas as pd
import numpy as np


In [2]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(data_id=31, return_X_y=True)

In [3]:
X.isna().sum()

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
dtype: int64

In [4]:
X  

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,4,real estate,67,none,own,2,skilled,1,yes,yes
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,2,real estate,22,none,own,1,skilled,1,none,yes
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,3,real estate,49,none,own,1,unskilled resident,2,none,yes
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,4,life insurance,45,none,for free,1,skilled,2,none,yes
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,4,no known property,53,none,for free,2,skilled,2,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12,existing paid,furniture/equipment,1736,<100,4<=X<7,3,female div/dep/mar,none,4,real estate,31,none,own,1,unskilled resident,1,none,yes
996,<0,30,existing paid,used car,3857,<100,1<=X<4,4,male div/sep,none,4,life insurance,40,none,own,1,high qualif/self emp/mgmt,1,yes,yes
997,no checking,12,existing paid,radio/tv,804,<100,>=7,4,male single,none,4,car,38,none,own,1,skilled,1,none,yes
998,<0,45,existing paid,radio/tv,1845,<100,1<=X<4,4,male single,none,4,no known property,23,none,for free,1,skilled,1,yes,yes


In [5]:
# Feature Selection

X = X.loc[:,['age','credit_amount','duration','num_dependents','credit_history','savings_status','job','housing']]

In [6]:
# Encoding and Scaling

from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, _encoders
#is ordinal the propoer encoder?

enc = LabelEncoder()
y = enc.fit_transform(y=y) 

enc = OrdinalEncoder()
X_slice = X.loc[:,['credit_history','savings_status','job','housing']]
for column in X_slice:
    X[column] = enc.fit_transform(X[[column]])

sca = StandardScaler()
X_slice = X.loc[:,['age','credit_amount','duration','num_dependents']]
for column in X_slice:
    X[column] = sca.fit_transform(X[[column]])

In [7]:
X

,age,credit_amount,duration,num_dependents,credit_history,savings_status,job,housing
0,2.766456,-0.745131,-1.236478,-0.428290,1.0,4.0,1.0,1.0
1,-1.191404,0.949817,2.248194,-0.428290,3.0,2.0,1.0,1.0
2,1.183312,-0.416562,-0.738668,2.334869,1.0,2.0,3.0,1.0
3,0.831502,1.634247,1.750384,2.334869,3.0,2.0,1.0,0.0
4,1.535122,0.566664,0.256953,2.334869,2.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...
995,-0.399832,-0.544162,-0.738668,-0.428290,3.0,2.0,3.0,1.0
996,0.391740,0.207612,0.754763,-0.428290,3.0,2.0,0.0,1.0
997,0.215835,-0.874503,-0.738668,-0.428290,3.0,2.0,1.0,1.0
998,-1.103451,-0.505528,1.999289,-0.428290,3.0,2.0,1.0,0.0


In [8]:
# Dataset splitting

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test, test_size=0.5, random_state=0)

models = {}


In [9]:
# model selection

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_neighbors': [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31]
    #'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
        }
model = KNeighborsClassifier(n_neighbors=3, metric='minkowski', p=2)
grid_clf = GridSearchCV(model, param_grid, cv=10)
grid_clf.fit(X_train, y_train)

model = grid_clf.best_estimator_
y_pred = model.predict(X_test)

from sklearn.metrics import recall_score
score = recall_score(y_test, y_pred)

print(model)
print(score)

models['KNN'] = model


KNeighborsClassifier(n_neighbors=9)
0.9295774647887324


/Users/koa/Documents/ML Course/credit_risks/.venv/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [10]:
# model selection 2

from sklearn.linear_model import LogisticRegression
param_grid = {
    'C': [0.01,0.1,0.3,0.5,0.8,1,1.5,2],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
        }
model2 = LogisticRegression(random_state=0)
grid_clf = GridSearchCV(model2, param_grid, cv=10)
grid_clf.fit(X_train, y_train)


model2 = grid_clf.best_estimator_
y_pred2 = model2.predict(X_test)

from sklearn.metrics import recall_score
score2 = recall_score(y_test, y_pred2)

print(model2)
print(score2)

models['LogisticRegression'] = model2

#print(model2.feature_names_in_,model2.coef_)



LogisticRegression(C=0.1, random_state=0, solver='liblinear')
0.9154929577464789


In [11]:
# model selection 3

from sklearn.svm import SVC
param_grid = {
    'C': [0.01,0.1,0.3,0.5,0.8,1,1.2,1.5,2],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [1,2,3,4]
        }
model3 = SVC(C=0.2)
grid_clf = GridSearchCV(model3, param_grid, cv=10)
grid_clf.fit(X_train, y_train)


model3 = grid_clf.best_estimator_
y_pred2 = model3.predict(X_test)



y_pred3 = model3.predict(X_test)

from sklearn.metrics import recall_score
score3 = recall_score(y_test, y_pred3)

print(model3)
print(score3)

models['SVC'] = model3

SVC(C=0.8, degree=4, kernel='poly')
0.9154929577464789


In [13]:
def export_to_pickle(models_dict,X_train, X_test, y_train, y_test, X, y):
    import pickle
    path = 'models'
    for key in models_dict.keys():
        pickle.dump(models_dict[key], open('models/'+key+'.pkl', 'wb'))

    dfs = [X_train, X_test, y_train, y_test, X, y]
    pickle.dump(dfs, open('models/dataframes.pkl', 'wb'))



export_to_pickle(models,X_train, X_test, y_train, y_test, X, y)